# Imports

In [1]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn as nn

import timm

from PIL import Image
import os
from contextlib import redirect_stdout
import ssl

import time

import matplotlib.pyplot as plt

ssl._create_default_https_context = ssl._create_unverified_context

# Constants

In [2]:
dataset_path = '/kaggle/input/final-stage-3-v3/Stage 3 V3'

# Dataset & Preprocessing

In [3]:
class ResizeWithPadding:
    def __init__(self, size):
        self.size = size

    def __call__(self, img):
        w, h = img.size
        if h > w:
            new_h = self.size
            new_w = int(w * (self.size / h))
        else:
            new_w = self.size
            new_h = int(h * (self.size / w))
        
        resized_img = img.resize((new_w, new_h), Image.LANCZOS)

        padded_img = Image.new("RGB", (self.size, self.size), color=(0, 0, 0))
        padded_img.paste(resized_img, ((self.size - new_w) // 2, (self.size - new_h) // 2))
        
        return padded_img

transform = transforms.Compose([
    ResizeWithPadding(256),
    transforms.ToTensor(),
])

if os.path.exists(dataset_path + '.DS_Store'):
        os.remove(dataset_path + '.DS_Store')

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

In [4]:
total_size = len(dataset)

train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# Loading & Modifying ResNet-152 Model

In [5]:
resnet_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)

with redirect_stdout(None):
    resnet_model.eval()

num_features = resnet_model.fc.in_features
num_classes = 2

new_fc_layers = [
    torch.nn.Linear(num_features, 512),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(512, 256),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(256, num_classes)
]
resnet_model.fc = torch.nn.Sequential(*new_fc_layers)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# Loading & Modifying InceptionV4 Model

In [6]:
inception_model = timm.create_model('inception_v4', pretrained=True)

num_features = inception_model.last_linear.in_features
num_classes = 2

new_fc_layers = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, num_classes)
)

inception_model.last_linear = new_fc_layers

# Hyperparameters

In [7]:
num_epochs = 150
batch_size = 8
patience = 15
criterion = torch.nn.CrossEntropyLoss()
resnet_optimizer = torch.optim.Adam(resnet_model.parameters(), lr=0.0001, weight_decay=1e-4)
inception_optimizer = torch.optim.Adam(inception_model.parameters(), lr=0.0001, weight_decay=1e-4)
resnet_model = resnet_model.to('cuda')
inception_model = inception_model.to('cuda')

# Training

In [8]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [9]:
def train(mod, name, opt):
    best_val_loss = float('inf')
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    sensitivities = []
    specificities = []
    
    epochs_no_improve = 0
    early_stop = False

    for epoch in range(num_epochs):
        start_time = time.time()
        mod.train()
        train_loss = 0
        correct_train = 0
        total_train = 0
        true_positives = 0
        true_negatives = 0
        false_positives = 0
        false_negatives = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to('cuda'), labels.to('cuda')
            opt.zero_grad()

            outputs = mod(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            opt.step()

            train_loss += loss.item()
            _, predicted_train = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted_train == labels).sum().item()

            true_positives += ((predicted_train == 1) & (labels == 1)).sum().item()
            true_negatives += ((predicted_train == 0) & (labels == 0)).sum().item()
            false_positives += ((predicted_train == 1) & (labels == 0)).sum().item()
            false_negatives += ((predicted_train == 0) & (labels == 1)).sum().item()

        train_loss /= len(train_loader)
        train_accuracy = 100 * correct_train / total_train

        sensitivity = true_positives / (true_positives + false_negatives)
        specificity = true_negatives / (true_negatives + false_positives)

        mod.eval()
        val_loss = 0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to('cuda'), labels.to('cuda')
                outputs = mod(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted_val = torch.max(outputs, 1)
                total_val += labels.size(0)
                correct_val += (predicted_val == labels).sum().item()

        val_loss /= len(val_loader)
        val_accuracy = 100 * correct_val / total_val

        flag = False
        if val_loss < best_val_loss:
            flag = True
            best_val_loss = val_loss
            torch.save(mod.state_dict(), f'best_{name}_model.pth')
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                early_stop = True

        end_time = time.time()
        epoch_time = end_time - start_time

        if flag:
            print(f'Epoch [{epoch+1}/{num_epochs}]: {epoch_time:.2f}s | Saved Model\nTrain Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.2f}%\nValidation Loss: {val_loss:.4f} | Validation Accuracy: {val_accuracy:.2f}%')
        else:
            print(f'Epoch [{epoch+1}/{num_epochs}]: {epoch_time:.2f}s\nTrain Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.2f}%\nValidation Loss: {val_loss:.4f} | Validation Accuracy: {val_accuracy:.2f}%')
        print('===================================================================================================')

        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)
        sensitivities.append(sensitivity)
        specificities.append(specificity)
        
        if early_stop:
            print('========================Early Stopped========================')
            break
        
    return train_losses, val_losses, train_accuracies, val_accuracies, sensitivities, specificities

def plot_graphs(train_losses, val_losses, train_accuracies, val_accuracies, sensitivities, specificities):
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Stage 3: Training and Validation Loss')
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 5))
    plt.plot(train_accuracies, label='Training Accuracy')
    plt.plot(val_accuracies, label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Stage 3: Training and Validation Accuracy')
    plt.legend()
    plt.show()

In [10]:
inception_train_losses, inception_val_losses, inception_train_accuracies, inception_val_accuracies, inception_sensitivities, inception_specificities = train(inception_model, 'inception', inception_optimizer)

Epoch [1/150]: 36.92s | Saved Model
Train Loss: 0.4406 | Train Accuracy: 80.97%
Validation Loss: 0.2340 | Validation Accuracy: 89.44%
Epoch [2/150]: 35.93s | Saved Model
Train Loss: 0.2998 | Train Accuracy: 88.86%
Validation Loss: 0.2214 | Validation Accuracy: 90.68%
Epoch [3/150]: 35.76s
Train Loss: 0.2640 | Train Accuracy: 91.11%
Validation Loss: 0.2606 | Validation Accuracy: 88.82%
Epoch [4/150]: 35.83s
Train Loss: 0.1812 | Train Accuracy: 93.89%
Validation Loss: 0.2239 | Validation Accuracy: 90.68%
Epoch [5/150]: 36.13s | Saved Model
Train Loss: 0.2126 | Train Accuracy: 92.58%
Validation Loss: 0.1707 | Validation Accuracy: 91.30%
Epoch [6/150]: 35.57s
Train Loss: 0.1592 | Train Accuracy: 94.74%
Validation Loss: 0.2933 | Validation Accuracy: 88.20%
Epoch [7/150]: 36.14s | Saved Model
Train Loss: 0.1311 | Train Accuracy: 95.59%
Validation Loss: 0.1683 | Validation Accuracy: 92.55%
Epoch [8/150]: 35.69s
Train Loss: 0.1686 | Train Accuracy: 94.28%
Validation Loss: 0.2605 | Validation A

KeyboardInterrupt: 

In [11]:
plot_graphs(inception_train_losses, inception_val_losses, inception_train_accuracies, inception_val_accuracies, inception_sensitivities, inception_specificities)

NameError: name 'inception_train_losses' is not defined

In [ ]:
resnet_train_losses, resnet_val_losses, resnet_train_accuracies, resnet_val_accuracies, resnet_sensitivities, resnet_specificities = train(resnet_model, 'resnet', resnet_optimizer)

In [ ]:
plot_graphs(resnet_train_losses, resnet_val_losses, resnet_train_accuracies, resnet_val_accuracies, resnet_sensitivities, resnet_specificities)

In [ ]:
def test(model, name):
    model.load_state_dict(torch.load(f'best_{name}_model.pth'))

    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to('cuda'), labels.to('cuda')
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_loss /= len(test_loader)
    test_accuracy = 100 * correct / total

    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

In [ ]:
test(inception_model, 'inception')

In [ ]:
test(resnet_model, 'resnet')

In [ ]:
def test_ensemble(models, names):
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to('cuda'), labels.to('cuda')
            ensemble_outputs = torch.zeros(inputs.size(0), num_classes).to('cuda')

            for model, name in zip(models, names):
                model.eval()
                outputs = model(inputs)
                ensemble_outputs += outputs

            ensemble_outputs /= len(models)

            loss = criterion(ensemble_outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(ensemble_outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_loss = total_loss / len(test_loader)
    test_accuracy = 100 * correct / total

    print(f'Ensemble Test Loss: {test_loss:.4f}, Ensemble Test Accuracy: {test_accuracy:.2f}%')
    

In [ ]:
test_ensemble([inception_model, resnet_model], ['inception', 'resnet'])